![rwth_imb_logo.png](attachment:rwth_imb_logo.png)

[Previous notebook: 2_1_pullout_from_rigid_matrix](./2_1_pullout_from_rigid_matrix.ipynb)

[Next notebook: 2_3_pull_out_from_elastic_matrix_supported_at_loaded_end](./2_3_pull_out_from_elastic_matrix_supported_at_loaded_end.ipynb)

# Example 2.2: Pullout from rigid matrix
## BC1, BC2, BC3 and free length
This example shows the analytically solved model of a pullout test with free-length of a bar 

In [ ]:
%reset -f
import sympy as sp
import numpy as np
from IPython.display import display
from sympy.plotting import plot as splot, plot3d as splot3d
sp.init_printing()

## Variables involved in the boundary value problem
#### Geometrical parameters
-  $A_\mathrm{f}$ | Young's modulus of the reinforcement
-  $p$: Perimiter of the reinforcement
-  $L_\mathrm{f}$: Free length of the pulled-out bar

#### Coordinate variable
-  $x$: Longitudinal coordinate

#### Material parameters
-  $E_\mathrm{f}$: Young's modulus of the reinforcement
-  $\tau$: Frictional bond stress

#### Control parameter
-  $P, w$: Pullout force, pullout displacement

#### State parameter
-  $a$: Length of the debonded zone

#### Integration constants
-  $A,B,C,D$: Integration constants to be resolved through boundary and continuity conditions

In [ ]:
E_f, A_f, tau, p = sp.symbols('E_f, A_f, tau, p')
A, B = sp.symbols('A, B')
C, D = sp.symbols('C, D')
P, w = sp.symbols('P, w')
x, a, L_b, L_f = sp.symbols('x, a, L_b, L_f')

## Free length: equilibrium, elasticity and kinematics  
In the domain 1 with $x \in (0, L_\mathrm{f})$ we can define the following differential equations:<br>
__Equilibrium:__ Consider the equilibrium equation along the free length of the bar
\begin{align}
\frac{\partial \sigma_\mathrm{f1}(x)}{\partial x} &= 0.
\end{align}
By integrating this equation we obtain the stress as an unknown constant
\begin{align}
\sigma_\mathrm{f1}(x) &= A.
\end{align}
__Elasticity:__ Assuming linear alastic behavior with the Young's modulus $E_\mathrm{f}$ we obtain the strain $\varepsilon$ as
\begin{align}
\varepsilon_\mathrm{f1}(x) = \frac{A}{E_\mathrm{f}}.
\end{align}
__Kinematics:__ The displacement of the bar within the free length is obtained as an integral
\begin{align}
u_\mathrm{f1}(x) = \int \varepsilon_\mathrm{f}(x) \; \mathrm{d}x = \frac{A}{E_\mathrm{f}}x + B
\end{align}

In [ ]:
d_sig_f1 = 0
sig_f1 = sp.integrate(d_sig_f1, x) + A
eps_f1 = sig_f1 / E_f
u_f1 = sp.integrate(eps_f1, x) + B
sig_f1, eps_f1, u_f1 # check

## Embedded length: equilibrium, elasticity and compatibility 
In the halfspace denoted $a$ with $x \in (-\infty, 0)$ the governing equations take the folowing form:<br>
__Equilibrium__: The equilibrium equation along the free length of the bar introduces the equivalence between the normal force in the reinforcement $\sigma_\mathrm{f} A_\mathrm{f}$ and and the bond intensity within an ifinitesimal element $\mathrm{d}x$
\begin{align}
\frac{\partial \sigma_\mathrm{fa}}{\partial x} &= \frac{p \tau}{A_\mathrm{f}}.
\end{align}
By integrating this equation we obtain the stress as an unknown constant
\begin{align}
\sigma_\mathrm{fa}(x) &=  \frac{p \tau}{A_\mathrm{f}} x + C.
\end{align}
__Elasticity__: Assuming linear elastic behavior with the Young's modulus $E_\mathrm{f}$ we obtain the strain $\varepsilon$ as
\begin{align}
\varepsilon_\mathrm{fa}(x) = \frac{1}{E_\mathrm{f}} \left(\frac{p \tau}{A_\mathrm{f}} x + C \right).
\end{align}
__Kinematics__: The displacement of the bar within the free length is obtained as an integral
\begin{align}
u_\mathrm{fa}(x) = 
\frac{p \tau x^{2}}{2 A_\mathrm{f} E_\mathrm{f}} + \frac{C x}{E_{\mathrm{f}}} + D 
\end{align}

In [ ]:
d_sig_fa = p * tau / A_f
sig_fa = sp.integrate(d_sig_fa, x) + C
eps_fa = sig_fa / E_f
u_fa = sp.integrate(eps_fa, x) + D
sig_fa, eps_fa, u_fa # check

### Resolving for integration constants (step 1)
__Condition 1__: Stress in the free length must be equal to $\sigma(L_\mathrm{f1}) = P/A_\mathrm{f}$.<br>
__Condition 2__: Displacement at the transition from embedded to the free zone $x-0$ must be equal for embedded and free displacement fields $u_\mathrm{f1}(0) = u_\mathrm{fa}(0)$.<br>
__Condition 3__: Displacement at the transition from embedded to the free zone $x-0$ must be equal for embedded and free displacement fields $\varepsilon_\mathrm{f1}(0) = \varepsilon_\mathrm{fa}(0)$.<br>
These three conditions can be used to resolve three integration constants. Instead of doing it manually, let us used the sympy solve method to get the results just by pushing shift+return above the next cell. 

In [ ]:
eqns_ABC = {sig_f1 - P / A_f,
            u_f1.subs({x:0}) - u_fa.subs({x:0}),
            eps_f1.subs({x:0}) - eps_fa.subs({x:0})}
ABC_subs = sp.solve(eqns_ABC,{A,B,C})
ABC_subs # display the result

Verify the result by inserting $A,B,C$ into the original equations for $u_\mathrm{f}, \varepsilon_\mathrm{f}$ and $\sigma_\mathrm{f}$.
### Resolving for integration constants (step 2)
The solution for the integration constant $D$ requires a second thought. Can we say something about how does the displacement approach zero within the embedded length? 

__Condition 4__: We can postulate, that at some unkonwn distance $a$, the slip between the reinforcement and the matrix will be zero, i.e. $u_\mathrm{fa}(a) = 0$.

However, this postulate does not solve the problem. This additional condition has introduced an additional uknown parameter $a$ so that the problem is still not solvable and another boundary or continuity condition needs to be found.

__Condition 5__: We futher postulate, that also the strain $varepsilon$ vanishes at the same distance $a$ as displacement reaches zero. i.e. $\varepsilon_\mathrm{fa}(a) = 0$.

By employing these conditions, we can resolve for the two uknown parameters $D$ and $a$ using sympy.solve method.

In [ ]:
eqns_aD = {eps_fa.subs(ABC_subs).subs({x:a}),
           u_fa.subs(ABC_subs).subs({x:a})}
aD_subs = sp.solve(eqns_aD,{a,D})[0]
aD_subs

With the known values of integration parameters we can resolve all the fields. Using the Heaviside step function $\theta(x)$ we can conveniently obtain a single function covering the domain of $x$ piece by piece.
\begin{align}
  u_\mathrm{f} & = \left\{
  \begin{array}{ll}
  u_\mathrm{f1}(x) & \iff x > 0, \\
  u_\mathrm{fa}(x) & \iff x < 0 \land x > a, \; \mathrm{where} \; a = -\frac{P}{p\tau}, \\
  0      & \mathrm{otherwise}
  \end{array}
  \right.
\end{align}
Using the Heaviside function, we can write this in a more compact way as folows
\begin{align}
  u_\mathrm{f} & = u_\mathrm{f1}(x) \, \theta(x) + u_\mathrm{fa} \, \theta(-x) \, \theta(x - a)
\end{align}

In [ ]:
u_f_x = (u_f1.subs(ABC_subs).subs(aD_subs) * sp.Heaviside(x) +
         u_fa.subs(ABC_subs).subs(aD_subs) * sp.Heaviside(-x) * sp.Heaviside(x-aD_subs[a]))
eps_f_x = sp.diff(u_f_x,x)
sig_f_x = 1 / E_f * eps_f_x

## Problem solved - lets get the results
Define concrete values of the model parameters 

In [ ]:
data_f = dict(L_b=1, L_f = 1, p = 1, E_f = 1, A_f = 1, tau = 1, P_max=1)

In [ ]:
u_f_x_data = u_f_x.subs(data_f)
eps_f_x_data = eps_f_x.subs(data_f)
sig_f_x_data = sig_f_x.subs(data_f)
P_levels = np.linspace(0,data_f['P_max'],8)
L_bb = aD_subs[a].subs(data_f).subs({P:data_f['P_max']})
L_bb

In [ ]:
u_f_list = [u_f_x_data.subs({P: Pl}) for Pl in P_levels] + [(x,L_bb,data_f['L_f'])]
splot(*u_f_list, line_color='red')
eps_f_list = [eps_f_x_data.subs({P: Pl}) for Pl in P_levels] + [(x,L_bb,data_f['L_f'])]
splot(*eps_f_list, line_color='green')
sig_f_list = [sig_f_x_data.subs({P: Pl}) for Pl in P_levels] + [(x,L_bb,data_f['L_f'])]
splot(*sig_f_list, line_color='blue')

In [ ]:
P_push, P_pull = sp.solve(u_f_x.subs({x:L_f})-w, P)
sp.simplify(P_push), sp.simplify(P_pull)

In [ ]:
PW_push = P_push.subs(data_f)
PW_pull = P_pull.subs(data_f)
PW_push, PW_pull

In [ ]:
splot(PW_pull, (w,0,1), line_color='green')

## Introduce finite embedded length

In [ ]:
P_max = sp.solve(aD_subs[a] + L_b, P)[0]
w_argmax = sp.solve(P_max - P_pull, w)[0]
w_argmax

In [ ]:
P_pull_Lb = sp.Heaviside(w_argmax - w) * P_pull + sp.Heaviside(w - w_argmax) * P_max
PW_pull_Lb = P_pull_Lb.subs(data_f)
PW_pull_Lb

In [ ]:
splot(PW_pull_Lb, (w,0,1), line_color='green')

## Clamp the reinforcement

In [ ]:
d_P_dw = sp.diff(P_pull,w)
K_c = d_P_dw.subs({w:w_argmax})
K_c

In [ ]:
P_pull_clamped = (sp.Heaviside(w_argmax - w) * P_pull + 
       sp.Heaviside(w - w_argmax) * (P_max + K_c * (w - w_argmax)))
PW_pull_clamped = P_pull_clamped.subs(data_f)

In [ ]:
splot(PW_pull_clamped, (w,0,1), line_color='green')